# Real-Life experiment on the TCLab platform

## Import libs

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
from scipy.optimize import minimize

import tclab
import time

import package_DBR
from package_LAB import *
from package_DBR import *

## Experiment params

In [ ]:
TSim = 2200
Ts = 1.0

# Initial working points 
MV0 = 50
DV0 = 50

# Paths
SPPath = {0: 75, 800: 65}
DVPath = {0: DV0, 1500: DV0+5}
manPath = {0: True, 750 : False, TSim: False}
manMVPath = {0: MV0, TSim: MV0}

# Parameters for input-output dynamics
Kp = 0.57
T1p = 172.4
T2p = 2.8
Thetap = 2.0

# Parameters for disturbance dynamics
Kd = 0.29
T1d = 127.9
T2d = 21.8
Thetad = 12.4

# Parameters for the PID controller 
alpha = 0.5
gamma = 0.4
Tc = gamma * T1p
discretisation = ["TRAP", "TRAP"]


## Determine PID gains with IMC tuning

In [ ]:
Kc = 0
Ti = 0
Td = 0

Kc,Ti,Td = IMC_Tuning(Kp, Tc, Thetap, T1p, T2p)
print("Kc = {}, ti = {}, td = {}".format(Kc, Ti, Td))

## Experiment

In [ ]:
PV = []
MV = {"MV" : [], "MVp" : [], "MVi": [],  "MVd": [], "E" : []}

N = int(TSim/Ts)+1

t = []
SP = []
DV = []

Man = []
MVMan = []
MVFF = []

MVFFDelay = []
MVFFLL1 = []

MVDelay = []
PV1p = []
PV2p = []
ActivateFF = True

DVDelay = []
PV1d = []
PV2d = []

# Initialize TCLab
lab = tclab.TCLab()
PV0 = lab.T1

fig, (ax1, ax2, ax3) = plt.subplots(3, 1)
fig.set_figheight(12)
fig.set_figwidth(22)

l1, = ax1.step([0, 1], [0, 100], 'b-', linewidth=2, label='MV', where='post')
ax1.set_ylabel('Value of MV [%]')
ax1.set_title('Closed-Loop experiment on TCLab')
ax1.legend(loc='best')

l2, = ax2.step([0, 1], [0, 100], 'g-', linewidth=2, label='PV', where='post')
ax2.set_ylabel('Value of PV [°C]')
ax2.legend(loc='best')

l3, = ax3.step([0, 1], [0, 100], 'r-', linewidth=2, label='DV', where='post')
ax3.set_xlabel('Time [s]')
ax3.set_ylabel('Value of DV [%]')
ax3.legend(loc='best')



for i in range(0, N):
    
    t.append(i*Ts)

    # Input signal SP and DV
    SelectPath_RT(SPPath,t,SP)
    SelectPath_RT(DVPath,t,DV)

    # FeedForward
    Delay_RT(DV-DV0*np.ones_like(DV), np.max([Thetad-Thetap,0]),Ts,MVFFDelay)
    LeadLag_RT(MVFFDelay,-Kd/Kp, T1d, T1p, Ts, MVFFLL1)
    if ActivateFF:
        LeadLag_RT(MVFFLL1, 1, T2d, T2p, Ts, MVFF)
    else:
        #LeadLag_RT(MVFFLL1, 0, T2d, T2p, Ts, MVFF)
        MVFF.append(0)
    
    # PID controller mode
    SelectPath_RT(manPath,t,Man)
    SelectPath_RT(manMVPath, t, MVMan)
    
    # PID controller
    PID_RT(PV, SP[i], MV, Ts, Kc, Ti, Td, alpha, discretisation, PVinit=PV0, man=Man[i], manMV=MVMan, FF_MV=MVFF)
    # Process Dynamics
    lab.Q1(MV["MV"][-1])


    # Disturbance dynamics
    lab.Q2(DV[-1])

    PV.append(lab.T1)
    #print("PV2p[-1]= {}, PV2d[-1] = {}, PV0 = {}, Kp = {}, MV0 = {}".format(PV2p[-1],PV2d[-1] ,PV0 , Kp , MV0))


    # Display graphs
    l1.set_data(t, MV["MV"])
    l2.set_data(t, PV)
    l3.set_data(t, DV)

    clear_output(wait=True)
    
    ax1.set_xlim(0, t[-1]+1)
    ax2.set_xlim(0, t[-1]+1)
    ax3.set_xlim(0, t[-1]+1)

    if i > 1:
        ax1.set_ylim(myRound(np.min(MV["MV"]), 5)-5, myRound(np.max(MV["MV"]), 5)+5)
        ax2.set_ylim(myRound(np.min(PV), 5)-5, myRound(np.max(PV), 5)+5)
        ax3.set_ylim(myRound(np.min(DV), 5)-5, myRound(np.max(DV), 5)+5)

    display(fig)

    time.sleep(Ts)


f = plt.figure(0)
f.set_figwidth(15)
f.set_figheight(10)
plt.subplot(2,1,1)
plt.step(t, np.array(SP),'c-', label='Setpoints',where='post')
plt.step(t, np.array(PV), label='PV')
plt.legend(loc='best')
s = plt.figure(1)
s.set_figwidth(15)
s.set_figheight(10)
plt.subplot(2,1,2)
plt.step(t, np.array(MVFF),'m', label='FF MV')
plt.step(t, np.array(Man), label='Man MV')
plt.legend(loc='best')
s = plt.figure(2)
s.set_figwidth(15)
s.set_figheight(10)
plt.step(t, np.array(DV),'g', label='DV')
plt.step(t, np.array(MV["MV"]),'r', label='PID MV', where='post')

plt.legend(loc='upper right')

plt.show()